<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr2/blob/main/tetrabot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загружаем библиотеки, подключаем гугл диск

In [1]:
!pip install SpeechRecognition
!pip install fasttext
import requests
# from urllib.parse import urlencode
# import json
import gdown
import os
import zipfile
import numpy as np
import pandas as pd
from google.colab import drive
import fasttext
import speech_recognition as sr
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, concatenate, Input, CategoryEncoding, Normalization, Activation, Conv1D
from tensorflow.keras.optimizers import Adam, AdamW, RMSprop
from tensorflow.keras import utils
import tensorflow as tf


r = sr.Recognizer()
drive.mount('/content/gdrive')



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199773 sha256=6fcc9ef720b9689949584e4bc45837ec4b8ed524c4228e978e719e05f68c4731
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext
Mounted at /content/gdrive


In [2]:
ft = fasttext.load_model('/content/gdrive/MyDrive/AI_data/cc.ru.300.bin')


# Загружаем файл с разметкой

In [41]:

x_train = []

y_train_group = np.array([])
y_train_action = np.array([])

df_razmetka = pd.read_excel('/content/gdrive/MyDrive/AI_data/razmetka.xlsx')
for idx, row in df_razmetka.iterrows():
  x_train.append(ft.get_word_vector(row.text))
y_train_group = df_razmetka['group'].to_numpy()
y_train_action = df_razmetka['action'].to_numpy()

# Описываем и запускаем модель

In [94]:
trainY = tf.keras.utils.to_categorical(y_train_group)

x_train = tf.stack(x_train)

model = Sequential()
model.add(Dense(len(x_train[1]), activation='relu'))
model.add(Dense(800, activation='relu'))
model.add(Dense(400, activation='relu'))
model.add(Dense(trainY[0].shape[0], activation='softmax'))
from google.colab import drive
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, trainY, batch_size=128, epochs=150, verbose=1)




Epoch 1/150
1/1 [==============================] - 1s 1s/step - loss: 2.2952 - accuracy: 0.2262
Epoch 2/150
1/1 [==============================] - 0s 17ms/step - loss: 2.2189 - accuracy: 0.5833
Epoch 3/150
1/1 [==============================] - 0s 12ms/step - loss: 2.1312 - accuracy: 0.5833
Epoch 4/150
1/1 [==============================] - 0s 11ms/step - loss: 2.0176 - accuracy: 0.5833
Epoch 5/150
1/1 [==============================] - 0s 12ms/step - loss: 1.8727 - accuracy: 0.5833
Epoch 6/150
1/1 [==============================] - 0s 11ms/step - loss: 1.7031 - accuracy: 0.5833
Epoch 7/150
1/1 [==============================] - 0s 10ms/step - loss: 1.5383 - accuracy: 0.5833
Epoch 8/150
1/1 [==============================] - 0s 10ms/step - loss: 1.4217 - accuracy: 0.5833
Epoch 9/150
1/1 [==============================] - 0s 11ms/step - loss: 1.3686 - accuracy: 0.5833
Epoch 10/150
1/1 [==============================] - 0s 10ms/step - loss: 1.3286 - accuracy: 0.5833
Epoch 11/150
1/1 [===

# Описываем функцию распознавани аудио при помощи recognize_google

In [ ]:

def recognize_file_def(AudioFile):
  AudioFile = sr.AudioFile(AudioFile)
  with AudioFile as source:
      audio = r.record(source)
      # print(r.recognize_google(audio, language="ru-RU"))
      return r.recognize_google(audio, language="ru-RU")

# Перебираем файлы, распознаем текст

In [ ]:
df = pd.DataFrame(columns=["file", "text", "vector"])#, "text", "vector")

def search_all_files(directory):
  for filename in os.listdir(directory):
    if "chunk" in filename:
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
          if os.path.getsize(f)<90000000:
            recognize_file = recognize_file_def(f)
            # print(type(recognize_file))
            df.loc[ len(df.index )] = [f, recognize_file, ft.get_word_vector(recognize_file)]
        else:
          search_all_files(f)

search_all_files('/content/gdrive/MyDrive/AI_data/golos_blizko')
df.to_excel("/content/gdrive/MyDrive/AI_data/df.xlsx")

In [ ]:
print(type(ft.get_word_vector("Изменяй темп движения")))

<class 'numpy.ndarray'>


# Пропускаем распознанный текст  через FastText

In [ ]:
for row in df
x3_data500 = []
for row in data500.values:
  wordVector = ft.get_word_vector(str(row[1]))
  x3_data500.append(wordVector)
x3_data500 = np.array(x3_data500)